In [ ]:
import json
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


Reading the raw dataset.

In [ ]:
raw_directory = "../raw/article_bias_prediction"

data = []
for filename in os.listdir(raw_directory):
    filepath = os.path.join(raw_directory, filename)
    with open(filepath, "r") as file:
        data.append(json.load(file))

df = pd.DataFrame(data)
df


Dropping useless columns. The content column is
> the processed and tokenized content, which is used as input to the different models

and so the original content can be dropped.

In [ ]:
df = df.drop(
    columns=[
        "topic",
        "bias",
        "source",
        "url",
        "date",
        "authors",
        "content_original",
        "source_url",
        "ID",
    ]
)


Renaming and reordering columns.

In [ ]:
df = df.rename(columns={"content": "body", "bias_text": "leaning"})
df = df[["title", "body", "leaning"]].copy()


Categorizing the leaning label column.

In [ ]:
df["leaning"] = df["leaning"].astype("category")


Adding the politicalness label column.

In [ ]:
df["politicalness"] = pd.Categorical(["political"] * len(df))


Stripping the titles and the bodies.

In [ ]:
df["title"] = (df["title"].str.strip()).replace("", np.nan)
df["body"] = (df["body"].str.strip()).replace("", np.nan)


Dropping rows missing either the leaning or both the title and the body.

In [ ]:
df = df.dropna(subset=["leaning"])
df = df.dropna(subset=["title", "body"], how="all")


Printing body duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]


Dropping the body duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")


Inspecting the title and the body length.

In [ ]:
df["title_length"] = df["title"].fillna("").str.len()
df["title_word_count"] = df["title"].fillna("").str.split().str.len()
df["body_length"] = df["body"].fillna("").str.len()
df["body_word_count"] = df["body"].fillna("").str.split().str.len()
df = df.sort_values(by="body_length")
df.head()


After the inspection, rows with bodies shorter than 40 words seem to contain no political value. Removing them.

In [ ]:
body_word_count_lower_bound = 40
df = df[df["body_word_count"] >= body_word_count_lower_bound]


In [ ]:
df.to_parquet("../preprocessed/article_bias_prediction.parquet")
